<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/06%20-%20MNIST/01_MNIST(2_1ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

MNIST(손글씨 숫자 인식) 문제를 신경망으로 풀어보자!

참고\
https://www.tensorflow.org/tutorials/quickstart/beginner

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

## 데이터 정의

MNIST는 기본적으로 케라스 함수에서 불러올 수 있는데 그점을 이용하여 코딩을 할 것이다.

x_train의 크기는 [이미지 수, 세로 픽셀 수, 가로 픽셀 수] -> [60000, 28, 28]\
y_train의 크기는 [이미지 수] -> [60000]

y_train의 각 요소는 0~9이며 이전과 같은 one-hot 인코딩이 아닌 정수 인코딩으로 표현된다.\
 one-hot -- integer\
[0,0,0,0,1] -- [5]\
[0,1,0,0,0] -- [2]\
[0,0,0,1,0] -- [4]

때문에 one-hot 인코딩방식으로 전환하는 코드를 사용하였다.

In [ ]:
# MNIST 데이터셋을 케라스 함수로 받아온다.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 샘플 값을 정수에서 부동소수로 변환한다.
# 원래는 gray_color로 0~255사이의 값으로 정수로 저장되어 있는데 
# 모델 내에서 계산할 때에는 0~1 사이의 소수가 필요하기 때문이다.
# 동시에 [60000,28,28] 형태의 이미지를 [60000, 784]으로 flattening 한다.
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

# 레이블 데이터에 one-hot encoding을 적용한다.
y_train, y_test = tf.one_hot(y_train, depth=10).numpy(), tf.one_hot(y_test, depth=10).numpy()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
print(x_train[:3])
print(y_train[:3])
print(len(x_train))
print(len(y_train))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
60000
60000


## 모델 구성

### 신경망 구성

케라스 함수로 만든 layer함수의 API들은 여러 옵션을 통하여 편리성을 더한다.

기본적으로 가중치는 자동으로 초기화가 되며 사용자가 신경쓰지 않아도 되고 \
편향은 쓸지 안 쓸지의 여부만 옵션으로 전달해주는 등 \
잘 알고 쓰면 몇 줄 안되는 코드로 복잡한 신경망을 구성할 수 있다.

In [ ]:
# 텐서플로우의 케라서에서 제공하는 모델 함수를 사용하여 모델을 작성한다.
# 신경망의 레이어는 다음처럼 구성한다.
# 784(입력 특성값)
#   -> 256 (히든레이어 뉴런 갯수) -> 256 (히든레이어 뉴런 갯수)
#   -> 10 (결과값 0~9 분류)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation=tf.nn.relu, use_bias=False),  # 뉴런 수가 256개인 히든 레이어 생성
  tf.keras.layers.Dense(256, activation=tf.nn.relu, use_bias=False),
  tf.keras.layers.Dense(10, activation='softmax', use_bias=False)     # 결과값이 10가지가 되도록 하는 레이어
])

### 최적화 / 손실 클래스 선택

In [ ]:
# 최적화 함수는 adam
# 손실함수는 one-hot 인코딩에 알맞은 비용함수 categorical_crossentropy를 사용한다.
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### 모델 훈련  및 평가

In [ ]:
# epoch 크기는 15, batch 크기는 100으로 한다.
epochs = 15
batch_size = 100

# 모델을 훈련한다.
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

# 모델을 테스트 해본다.
model.evaluate(x_test,  y_test, batch_size=batch_size, verbose=2)

Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 70us/sample - loss: 1.4657 - accuracy: 0.9961
Epoch 2/15
60000/60000 [==============================] - 4s 72us/sample - loss: 1.4660 - accuracy: 0.9955
Epoch 3/15
60000/60000 [==============================] - 4s 69us/sample - loss: 1.4651 - accuracy: 0.9966
Epoch 4/15
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4660 - accuracy: 0.9955
Epoch 5/15
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4651 - accuracy: 0.9965
Epoch 6/15
60000/60000 [==============================] - 4s 65us/sample - loss: 1.4640 - accuracy: 0.9974
Epoch 7/15
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4647 - accuracy: 0.9968
Epoch 8/15
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4650 - accuracy: 0.9963
Epoch 9/15
60000/60000 [==============================] - 4s 65us/sample - loss: 1.4646 - accuracy: 0.9968
Epoch 10/15
60

[1.4798706030845643, 0.9812]

# 전체 코드

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

y_train, y_test = tf.one_hot(y_train, depth=10).numpy(), tf.one_hot(y_test, depth=10).numpy()

#########
# 신경망 모델 구성
######

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation=tf.nn.relu),  # 뉴런 수가 256개인 히든 레이어 생성
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation='softmax')     # 결과값이 10가지가 되도록 하는 레이어
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#########
# 신경망 모델 학습
######

epochs = 15
batch_size = 100

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

#########
# 결과 확인
######

model.evaluate(x_test,  y_test, batch_size=batch_size, verbose=2)


Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 70us/sample - loss: 0.2451 - accuracy: 0.9281
Epoch 2/15
60000/60000 [==============================] - 4s 65us/sample - loss: 0.0903 - accuracy: 0.9724
Epoch 3/15
60000/60000 [==============================] - 4s 64us/sample - loss: 0.0607 - accuracy: 0.9809
Epoch 4/15
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0439 - accuracy: 0.9861
Epoch 5/15
60000/60000 [==============================] - 4s 64us/sample - loss: 0.0318 - accuracy: 0.9900
Epoch 6/15
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0255 - accuracy: 0.9915
Epoch 7/15
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0229 - accuracy: 0.9929
Epoch 8/15
60000/60000 [==============================] - 4s 68us/sample - loss: 0.0162 - accuracy: 0.9948
Epoch 9/15
60000/60000 [==============================] - 4s 68us/sample - loss: 0.0162 - accuracy: 0.9947
Epoch 10/15
60

[0.11410180909082257, 0.9772]